## Starting folder
Please go `src/` folder using `cd`.

In [ ]:
cd src/

/content/drive/MyDrive/Research_MIPT/Codes/cms-scripts/a2f/src


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Research_MIPT/Codes/cms-scripts/a2f/src

/content/drive/MyDrive/Research_MIPT/Codes/cms-scripts/a2f/src


# Getting work

# Manual

## QE outputs parsing

In [4]:
ls

constants.py      main.py      __pycache__/  sc_a2f.py     sc_e.py    tc.py
interpolators.py  plotters.py  qe_ouputs.py  sc_direct.py  system.py  utils.py


In [3]:
from qe_outputs import Folder, DynElph, PhOuts

ModuleNotFoundError: ignored